In [8]:
import os

from sparse_autoencoder import (
    ActivationResamplerHyperparameters,
    AutoencoderHyperparameters,
    Hyperparameters,
    LossHyperparameters,
    Method,
    OptimizerHyperparameters,
    Parameter,
    PipelineHyperparameters,
    SourceDataHyperparameters,
    SourceModelHyperparameters,
    SweepConfig,
    sweep,
)

In [9]:
!source ../.env
os.environ["WANDB_NOTEBOOK_NAME"] = "gpt-2.ipynb"
def train_gpt_small_mlp_layers(
    expansion_factor: int = 4,
    layer: int = 6,
) -> None:
    """Run a new sweep experiment on GPT 2 Small's MLP layers.

    Args:
        expansion_factor: Expansion factor for the autoencoder.
        n_layers: Number of layers to train on. Max is 12.

    """
    sweep_config = SweepConfig(
        parameters=Hyperparameters(
            loss=LossHyperparameters(
                l1_coefficient=Parameter(max=0.03, min=0.008),
            ),
            optimizer=OptimizerHyperparameters(
                lr=Parameter(max=0.001, min=0.00001),
            ),
            source_model=SourceModelHyperparameters(
                name=Parameter("gpt2"),
                cache_names=Parameter(
                    [f"blocks.{layer}.hook_mlp_out"]
                ),
                hook_dimension=Parameter(768),
            ),
            source_data=SourceDataHyperparameters(
                dataset_path=Parameter("alancooney/sae-monology-pile-uncopyrighted-tokenizer-gpt2"),
                context_size=Parameter(256),
                pre_tokenized=Parameter(value=True),
                pre_download=Parameter(value=False),  # Default to streaming the dataset
            ),
            autoencoder=AutoencoderHyperparameters(
                expansion_factor=Parameter(value=expansion_factor)
            ),
            pipeline=PipelineHyperparameters(
                max_activations=Parameter(1_000_000_000),
                checkpoint_frequency=Parameter(100_000_000),
                validation_frequency=Parameter(100_000_000),
                max_store_size=Parameter(1_000_000),
            ),
            activation_resampler=ActivationResamplerHyperparameters(
                resample_interval=Parameter(200_000_000),
                n_activations_activity_collate=Parameter(100_000_000),
                threshold_is_dead_portion_fires=Parameter(1e-6),
                max_n_resamples=Parameter(4),
            ),
        ),
        method=Method.RANDOM,
    )

    sweep(sweep_config=sweep_config)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
train_gpt_small_mlp_layers()


Create sweep with ID: uzdfnotv
Sweep URL: https://wandb.ai/shahrad_m/sparse-autoencoder/sweeps/uzdfnotv


wandb: Agent Starting Run: e2xzdoaj with config:
wandb: 	activation_resampler: {'max_n_resamples': 4, 'n_activations_activity_collate': 100000000, 'resample_dataset_size': 819200, 'resample_interval': 200000000, 'threshold_is_dead_portion_fires': 1e-06}
wandb: 	autoencoder: {'expansion_factor': 4}
wandb: 	loss: {'l1_coefficient': 0.01862974667157214}
wandb: 	optimizer: {'adam_beta_1': 0.9, 'adam_beta_2': 0.99, 'adam_weight_decay': 0, 'amsgrad': False, 'fused': False, 'lr': 0.0001223696999119536}
wandb: 	pipeline: {'checkpoint_frequency': 100000000, 'log_frequency': 100, 'max_activations': 1000000000, 'max_store_size': 1000000, 'num_workers_data_loading': 0, 'source_data_batch_size': 16, 'train_batch_size': 8192, 'validation_frequency': 100000000, 'validation_n_activations': 8192}
wandb: 	random_seed: 49
wandb: 	source_data: {'context_size': 256, 'dataset_column_name': 'input_ids', 'dataset_path': 'alancooney/sae-monology-pile-uncopyrighted-tokenizer-gpt2', 'pre_download': False, 'pre_t

/home/mila/s/shahrad.mohammadzadeh/projects/pythia-hallucination/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2 into HookedTransformer


Activations trained on:   1%|          | 6995968/1000000000 [22:36<57:33:25, 4792.36it/s, stage=train]     wandb: Ctrl + C detected. Stopping sweep.


train/learned_activations_l0_norm,█▃▂▂▁▁▁▁
train/learned_neuron_activity/alive_over_8192_activations,█▃▁▃▇▆▆▄
train/learned_neuron_activity/alive_over_98304_activations,▁█
train/learned_neuron_activity/almost_dead_1.0e-05_over_8192_activations,▁▆█▆▂▃▃▅
train/learned_neuron_activity/almost_dead_1.0e-05_over_98304_activations,█▁
train/learned_neuron_activity/almost_dead_1.0e-06_over_8192_activations,▁▆█▆▂▃▃▅
train/learned_neuron_activity/almost_dead_1.0e-06_over_98304_activations,█▁
train/learned_neuron_activity/dead_over_8192_activations,▁▆█▆▂▃▃▅
train/learned_neuron_activity/dead_over_98304_activations,█▁
train/loss/l2_reconstruction_loss,▆█▆▅▅▅▅▁
train/loss/learned_activations_l1,▅▁▁▂▂▄▇█
